# Tutorial PAMAP2 with mcfly

The goal of this tutorial is to get you familiar with training Neural Networks for time series using mcfly. At the end of the tutorial, you will have compared several Neural Network architectures you know how to train the best performing network.

As an example dataset we use the publicly available [PAMAP2 dataset](https://archive.ics.uci.edu/ml/datasets/PAMAP2+Physical+Activity+Monitoring). It contains time series data from movement sensors worn by nine individuals. The data is labelled with the activity types that these individuals did and the aim is to train and evaluate a *classifier*.

Before you can start, please make sure you install mcfly (see the [mcfly installation page](https://github.com/NLeSC/mcfly)).

## Import required Python modules

In [1]:
import sys
import os
import json
import numpy as np
import pandas as pd
import mcfly
from mcfly.find_architecture import train_models_on_samples
from mcfly.find_architecture import find_best_architecture
from sklearn.model_selection import train_test_split
# from sktime.datasets import load_from_tsfile
import tensorflow as tf
import warnings
import dill

warnings.filterwarnings('ignore')
np.random.seed(2)

In [2]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [ ]:
# deprecated
# sys.path.insert(0, os.path.abspath('../..'))
# from utils import tutorial_pamap2

## Download data pre-procesed data

We have created a function for you to fetch the preprocessed data from https://zenodo.org/record/834467. Please specify the `directory_to_extract_to` in the code below and then execute the cell. This will download the preprocessed data into the directory in the `data` subdirectory. The output of the function is the path where the preprocessed data was stored.

In [3]:
# Specify in which directory you want to store the data:
directory_to_extract_to = '../'

In [4]:
!ls -l ../data/PAMAP2/PAMAP2/preprocessed

Der Befehl "ls" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [4]:
# data_path = tutorial_pamap2.download_preprocessed_data(directory_to_extract_to)
# data_path = '../data/PAMAP2/PAMAP2/preprocessed'
data_path = '../data/PAMAP2/PAMAP2/preprocessed/'

## A bit about the data

The [PAMAP2 dataset](https://archive.ics.uci.edu/ml/datasets/PAMAP2+Physical+Activity+Monitoring) contains data from three movement sensors worn by nine test subjects. These subjects performed a protocol of several activities.

The data originates from three sensors (on the hand, ankle and chest) and from each of the sensors we have three channels (acceleration on x, y and z axes). This gives us, for each time step, 9 values. The data is recorded on 100 Hz.

The preprocessed data is split into smaller segments with a window of 512 time steps, corresponding to 5.12 seconds. We only include segments that completely fall into one activity period: the activity is the *label* of the segment.

The goal of classification is to assign an activity label to an previously unseen segment.

## Load the pre-processed data

Load the preprocessed data as stored in Numpy-files. Please note that the data has already been split up in a training (training), validation (val), and test subsets. It is common practice to call the input data X and the labels y.

In [ ]:
X = np.load(f'{data_path}/preprocessed.npy')
targets = pd.read_csv(f'{data_path}/profile_targets.csv')

In [5]:
X = np.swapaxes(X, 1, 2)
X.shape

(2205, 60, 17)

In [6]:
targets.head()

,target_cooler_condition,target_valve_condition,target_internal_pump_leakage,target_hydraulic_accumulator_pressure,target_stable_flag
0,3,100,0,130,1
1,3,100,0,130,1
2,3,100,0,130,1
3,3,100,0,130,1
4,3,100,0,130,1


In [7]:
targets['target_cooler_condition'].value_counts()

100    741
3      732
20     732
Name: target_cooler_condition, dtype: int64

In [8]:
codes, uniques = targets['target_cooler_condition'].factorize()
y_binary = tf.keras.utils.to_categorical(codes, num_classes=len(uniques))

In [9]:
X_train, X_test, y_train_binary, y_test_binary = train_test_split(X, y_binary, test_size=0.10, random_state=42)

X_train.shape, X_test.shape, y_train_binary.shape, y_test_binary.shape


((1984, 60, 17), (221, 60, 17), (1984, 3), (221, 3))

In [10]:
X_train, X_val, y_train_binary, y_val_binary = train_test_split(X_train, y_train_binary, test_size=0.20, random_state=42)

X_train.shape, X_val.shape, y_train_binary.shape, y_val_binary.shape

((1587, 60, 17), (397, 60, 17), (1587, 3), (397, 3))

In [5]:
# X_train, y_train_binary, X_val, y_val_binary, X_test, y_test_binary, labels = tutorial_pamap2.load_data(data_path)

X_train = np.load(f'{data_path}/X_train.npy')
y_train_binary = np.load(f'{data_path}/y_train.npy')
X_val = np.load(f'{data_path}/X_val.npy')
y_val_binary = np.load(f'{data_path}/y_val.npy')
X_test = np.load(f'{data_path}/X_test.npy')
y_test_binary = np.load(f'{data_path}/y_test.npy')

In [6]:
with open(os.path.join(data_path, 'labels.json'), 'r') as fn:
    labels = json.load(fn)

Data X and labels y are of type Numpy array. In the cell below we inspect the shape of the data. As you can see the shape of X is expressed as a Python tuple containing: the number of samples, length of the time series, and the number of channels for each sample. Similarly, the shape of y is represents the number of samples and the number of classes (unique labels). Note that y has the format of a binary array where only the correct class for each sample is assigned a 1. This is called one-hot-encoding.

In [7]:
print('x shape:', X_train.shape)
print('y shape:', y_train_binary.shape)

x shape: (11397, 512, 9)
y shape: (11397, 7)


In [ ]:
# reshape ts file to fit the mcfly input shape requirement 
# xs = X_train.shape
# X_train.reshape((xs[0], xs[2],xs[1])).shape
# Output: (11397, 9, 512)

The data is split between train test and validation.

In [8]:
print('train set size:', X_train.shape[0])
print('validation set size:', X_val.shape[0])
print('test set size:', X_test.shape[0])

train set size: 11397
validation set size: 100
test set size: 1000


Let's have a look at the distribution of the labels:

In [ ]:
# frequencies = y_train_binary.mean(axis=0)
# frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
# frequencies_df

### *Question 1: How many channels does this dataset have?*
### *Question 2: What is the least common activity label in this dataset?*

    

## Generate models

First step in the development of any deep learning model is to create a model architecture. As we do not know what architecture is best for our data we will create a set of random models to investigate which architecture is most suitable for our data and classification task. This process, creating random models, checking how good they are and then selecting the best one is called a 'random search'. A random search is considered to be the most robust approach to finding a good model. You will need to specificy how many models you want to create with argument 'number_of_models'. See for a full overview of the optional arguments the function documentation of modelgen.generate_models by running `modelgen.generate_models?`.

##### What number of models to select?
This number differs per dataset. More models will give better results but it will take longer to evaluate them. For the purpose of this tutorial we recommend trying only 2 models to begin with. If you have enough time you can try a larger number of models, e.g. 10 or 20 models. Because mcfly uses random search, you will get better results when using more models.

In [21]:
num_classes = y_train_binary.shape[1]
metric = 'binary_accuracy' # 'accuracy'
models = mcfly.modelgen.generate_models(X_train.shape,
                                        number_of_classes=num_classes,
                                        number_of_models = 4,
                                        model_types=['InceptionTime'],
                                        metrics=[metric])

# Inspect the models
We can have a look at the models that were generated. The layers are shown as table rows. Most common layer types are 'Convolution' and 'LSTM' and 'Dense'. For more information see the [mcfly user manual](https://github.com/NLeSC/mcfly/wiki/User-manual) and the [tutorial cheat sheet](https://github.com/NLeSC/mcfly-tutorial/blob/master/cheatsheet.md). The summary also shows the data shape of each layer output and the number of parameters that are trained within this layer.

In [ ]:
models_to_print = range(len(models))
for i, item in enumerate(models):
    if i in models_to_print:
        model, params, model_types = item
        print("-------------------------------------------------------------------------------------------------------")
        print("Model " + str(i))
        print(" ")
        print("Hyperparameters:")
        print(params)
        print(" ")
        print("Model description:")
        model.summary()
        print(" ")
        print("Model type:")
        print(model_types)
        print(" ")

### *Question 3: Can you guess what hyperparameter 'learning rate' stands for?*

## Compare models
Now that the model architectures have been generated it is time to compare the models by training them on a subset of the training data and evaluating the models on the validation subset. This will help us to choose the best candidate model. The performance results for the models are stored in a json file, which we will visually inspect later on.

In [22]:
# Define directory where the results, e.g. json file, will be stored
resultpath = os.path.join(directory_to_extract_to, 'models')
if not os.path.exists(resultpath):
        os.makedirs(resultpath)

We are now going to train each of the models that we generated. On the one hand we want to train them as quickly as possible in order to be able to pick the best one as soon as possible. On the other hand we have to train each model long enough to get a good impression of its potential.

We can influence the train time by adjusting the number of data samples that are used. This can be set with the argument 'subset_size'. We can also adjust the number of times the subset is iterated over. This number is called an epoch. We recommend to start with no more than 5 epochs and a maximum subset size of 300. You can experiment with these numbers.

In [23]:
outputfile = os.path.join(resultpath, 'modelcomparison.json')
histories, val_accuracies, val_losses =\
    train_models_on_samples(X_train,
                            y_train_binary,
                            X_val,
                            y_val_binary,
                            models,
                            batch_size=5,
                            nr_epochs=20,
                            subset_size=1000,
                            verbose=True,
                            outputfile=outputfile,
                            metric=metric)

print('Details of the training process were stored in ', outputfile)

Generated models will be trained on subset of the data (subset size: 1000).
Training model 0 InceptionTime
Epoch 1/20
200/200 [==============================] - 12s 46ms/step - loss: 1.2852 - binary_accuracy: 0.8911 - val_loss: 11.8284 - val_binary_accuracy: 0.8186
Epoch 2/20
200/200 [==============================] - 9s 45ms/step - loss: 0.7483 - binary_accuracy: 0.9329 - val_loss: 0.4906 - val_binary_accuracy: 0.9643
Epoch 3/20
200/200 [==============================] - 9s 45ms/step - loss: 0.5516 - binary_accuracy: 0.9461 - val_loss: 0.4638 - val_binary_accuracy: 0.9729
Epoch 4/20
200/200 [==============================] - 9s 45ms/step - loss: 0.4893 - binary_accuracy: 0.9543 - val_loss: 0.4473 - val_binary_accuracy: 0.9657
Epoch 5/20
200/200 [==============================] - 9s 45ms/step - loss: 0.4115 - binary_accuracy: 0.9630 - val_loss: 0.5801 - val_binary_accuracy: 0.9771
Epoch 6/20
200/200 [==============================] - 9s 45ms/step - loss: 0.4067 - binary_accuracy: 0.961

### *Question 4: What does the term 'loss' in the output refer to?*

# Inspect model performance (Visualization)

We can inspect the learning process in the visualization tool on http://nlesc.github.io/mcfly/.

Alternatively, you can run the visualization from a local web service:
- Clone the mcfly github repository (if you haven't done so already for visualization)

 `git clone https://github.com/NLeSC/mcfly`


- navigate to the html folder:

 `cd mcfly/html`


- Start a web server. This can be done in various ways, for example:
 - for python 3 you can use: `python3 -m http.server`
 - for python 2 you can use: `python2 -m SimpleHTTPServer`

Notice the port number the web server is serving on. This is usually 8000.
With a web browser, navigate to [localhost:8000](localhost:8000). 

You need to upload the json file that contains the details of the training process. The following line of code shows the path to this file:

In [13]:
outputfile

'../models\\modelcomparison.json'

### *Question 5:  Look at the visualization. Which model performs best?*
### *Question 6:  Did you train all models with a sufficient number of iterations?*

# Inspect model performance (table)

Let's compare the performance of the models by showing the results as a table.

In [24]:
# metric = 'accuracy'
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_{}'.format(metric): [history.history[metric][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_{}'.format(metric): [history.history['val_{}'.format(metric)][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
modelcomparisons.to_csv(os.path.join(resultpath, 'modelcomparisons.csv'))

modelcomparisons

,model,train_binary_accuracy,train_loss,val_binary_accuracy,val_loss
0,"{'learning_rate': 0.016741735862847267, 'regul...",0.970286,0.297298,0.980000,0.387688
1,"{'learning_rate': 0.0024727913878189235, 'regu...",0.973571,0.303169,0.952857,0.775659
2,"{'learning_rate': 0.0001431293616306451, 'regu...",0.992571,0.114348,0.977143,0.277032
3,"{'learning_rate': 0.08286915344718057, 'regula...",0.948429,0.534275,0.961429,0.529184


In [ ]:
modelcomparisons.iloc[0,0]

# Choose the best model
Now that we found an effective architecture, we can choose the most promising model. For example, we can choose the model with the highest accuracy on the validation data set. To maximize this models performance, we will train this model on more data and more epochs.

In [26]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print('Model type and parameters of the best model:')
print("best_model_index:", best_model_index)
print(best_model_types)
print(best_params)

Model type and parameters of the best model:
best_model_index: 0
InceptionTime
{'learning_rate': 0.016741735862847267, 'regularization_rate': 0.00037365787711355855, 'network_depth': 5, 'filters_number': 92, 'max_kernel_size': 19}


## Train the best model on the full dataset

Now that we have identified the best model architecture out of our random pool of models we can continue by training the model on the full training set.

This would take some time, so instead we will train  on only a slightly larger subset.

In [28]:
#We make a copy of the model, to start training from fresh
nr_epochs = 5
datasize = X_train.shape[0] # Change in `X_train.shape[0]` if training complete data set

history = best_model.fit(
    X_train[:datasize,:,:], 
    y_train_binary[:datasize,:],
    epochs=nr_epochs, 
    validation_data=(X_val, y_val_binary)
)

Epoch 1/20
357/357 [==============================] - 60s 163ms/step - loss: 0.1477 - binary_accuracy: 0.9876 - val_loss: 0.0317 - val_binary_accuracy: 1.0000
Epoch 2/20
357/357 [==============================] - 58s 162ms/step - loss: 0.0873 - binary_accuracy: 0.9918 - val_loss: 0.0198 - val_binary_accuracy: 1.0000
Epoch 3/20
 13/357 [>.............................] - ETA: 55s - loss: 0.2132 - binary_accuracy: 0.9811

KeyboardInterrupt: 

### *Question 7: Why does it take longer to train the best model for one epoch now than when we were  comparing model archicatures earlier on?*

### *Question 8: Do you think it is useful to train with more than 1 epoch?*

### Saving, loading and comparing reloaded model with original model

The model can be saved for future use. The savemodel function will save two separate files: a json file for the architecture and a npy (numpy array) file for the weights.

In [29]:
modelname = 'my_bestmodel.h5'
model_path = os.path.join(resultpath,modelname)

In [30]:
best_model.save(model_path)

In [31]:
model_reloaded = tf.keras.models.load_model(model_path)

The model has been reloaded. Let's reassure that this model has the same weights

In [32]:
np.all([np.all(x==y) for x,y in zip(best_model.get_weights(), model_reloaded.get_weights())])

True

## Investigate model predictions

We will now dive further into the Neural network that we created.
We provide here a network that has been trained on the complete train set.

In [ ]:
model = tf.keras.models.load_model('./model/model.h5')

Note that the objects `models`, `best_model_fullytrained` and `best_model` that resulted from the mcfly functions are Keras objects. This means that you can use Keras functions on the objects, for example  `.predict`, (which when given the data, outputs the predictions for each sample) and `.evaluate` (which when given the data and the labels computes how well this model performs) . These functions are all documented in the [Keras documentation](https://www.tensorflow.org/api_docs/python/tf/keras). 

In [36]:
## Inspect model predictions on validation data
datasize = X_val.shape[0]
probs = best_model.predict(X_val[:datasize,:,:],batch_size=1)

100/100 [==============================] - 1s 6ms/step


Let's have a look at the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix)

In [37]:
#columns are predicted, rows are truth
predicted = probs.argmax(axis=1)
y_index = y_val_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
confusion_matrix

,lying,sitting,standing,walking,cycling,vaccuum_cleaning,ironing
lying,13,0,0,0,0,0,0
sitting,0,13,1,0,0,0,0
standing,0,0,20,0,0,0,0
walking,0,0,0,15,0,0,0
cycling,0,0,0,0,12,0,0
vaccuum_cleaning,0,0,0,0,0,8,0
ironing,0,0,0,0,0,0,18


In [39]:
## Test on Testset
score_test = best_model.evaluate(X_test, y_test_binary, verbose=True)
print('Score of best model')
print('Test loss:', score_test[0])
print('Test accuracy:', score_test[1])

32/32 [==============================] - 1s 44ms/step - loss: 0.1527 - binary_accuracy: 0.9871
Score of best model
Test loss: 0.15272612869739532
Test accuracy: 0.9871429204940796


In [38]:
## Inspect model predictions on validation data
datasize = X_test.shape[0]
probs = best_model.predict(X_test[:datasize,:,:],batch_size=1)

#columns are predicted, rows are truth
predicted = probs.argmax(axis=1)
y_index = y_test_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
confusion_matrix

1000/1000 [==============================] - 6s 6ms/step


,lying,sitting,standing,walking,cycling,vaccuum_cleaning,ironing
lying,153,0,0,0,0,5,0
sitting,0,134,6,0,0,0,11
standing,0,2,119,0,0,0,4
walking,0,0,0,165,0,0,0
cycling,0,0,0,1,101,2,5
vaccuum_cleaning,0,1,0,0,0,113,8
ironing,0,1,0,0,0,0,169
